# Setup

In [63]:
pip install scikit-learn

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.6 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 60.6/60.6 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.9 MB 2.6 MB/s eta 0:00:05
    --------------------------------------- 0.2/10.9 MB 2.6 MB/s eta 0:00:05
    --------------------------------------- 0.2/10.9 MB 1.8 MB/s eta 0:00:07
    --------------------------------------- 0.2/10.9 MB 1.8 MB/s eta 0:00:07
   - -------------------------------------- 0.3/10.9 MB 1.6 MB/s eta 0:00:07
   - -------------------------------------- 0.5/10.9 MB 2.0 MB/s eta 0:00:06
   -- ------------------------------------- 0.6/10.9 MB 2.2 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/10.9 MB 2.1 MB/s eta 0:00:05
   -- ---

In [64]:
# General
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

meter_file = 'src_data/_MeteringLineValue__202402102328.csv'
tarif_file = 'src_data/07-02-2024_2500_NP_99.csv'
output_dir_illustrations = './sink_illustrations/'
output_dir_data = './sink_illustrations/'
os.makedirs(output_dir_illustrations, exist_ok=True)
os.makedirs(output_dir_data, exist_ok=True)
meter_data = pd.read_csv(meter_file)
tarif_data = pd.read_csv(tarif_file)

# Exploration
Using data wrangler in VS Code for general exploration reg. distribution, nulls, types etc.

In [2]:
# Visualization setup
sns.set_theme(style="whitegrid")

# Histograms for numerical columns
numerical_cols_1 = meter_data.select_dtypes(include=[np.number]).columns
numerical_cols_2 = tarif_data.select_dtypes(include=[np.number]).columns

for col in numerical_cols_1:
    plt.figure(figsize=(8, 4))
    sns.histplot(meter_data[col].dropna(), kde=True)
    plt.title(f'Distribution of {col} in meter_data')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.savefig(os.path.join(output_dir_illustrations, f'meter_data_distribution_{col}.png'))
    plt.close()

for col in numerical_cols_2:
    plt.figure(figsize=(8, 4))
    sns.histplot(tarif_data[col].dropna(), kde=True)
    plt.title(f'Distribution of {col} in tarif_data')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.savefig(os.path.join(output_dir_illustrations, f'tarif_data_distribution_{col}.png'))
    plt.close()

# Count plots for categorical columns
categorical_cols_1 = meter_data.select_dtypes(include=[object]).columns
categorical_cols_2 = tarif_data.select_dtypes(include=[object]).columns

for col in categorical_cols_1:
    plt.figure(figsize=(8, 4))
    sns.countplot(y=meter_data[col])
    plt.title(f'Distribution of {col} in meter_data')
    plt.xlabel('Count')
    plt.ylabel(col)
    plt.savefig(os.path.join(output_dir_illustrations, f'meter_data_distribution_{col}.png'))
    plt.close()

for col in categorical_cols_2:
    plt.figure(figsize=(8, 4))
    sns.countplot(y=tarif_data[col])
    plt.title(f'Distribution of {col} in tarif_data')
    plt.xlabel('Count')
    plt.ylabel(col)
    plt.savefig(os.path.join(output_dir_illustrations, f'tarif_data_distribution_{col}.png'))
    plt.close()

# Main (explorative) results
## Meter Data
- Consumption
    - 'Id' -> each row with unique Id => no panel data
    - 'Value' -> looks like a standard distribution (bisschen linkschief)
- Time
    - 'TimestampUtc' 2023-10-12 -> to 2024-02-09 ; October to February
    - 'CreatedAt' -> similar to TimestampUtc ; 2023-10-14 to 2024-02-10
    - 'LastUpdatedAt' -> sometimes identical to 'CreatedAt'
- Other
    - 'MeteringLineId' -> Only one unique entry (FK?)
    - 'Origin' -> only value 'Getec'
    
## Tarif Data
- Pricing
    - 'TotalPrice_eur' -> coarsly 2 cluster (maybe 4 finer ones)
    - 'Grundpreis_eur_per_month' -> two clusters
    - 'Arbeitspreis_cent_per_kWh' -> looks like 3 standard deviation -> *clustering potentially interesting*
    - 'Consumption_kWh' -> perfectly flat distribution
    - 'PriceGuaranteeMonths' -> perfectly flat distribution
- Competitors
    - 'ProviderName' -> most common 'idealenergie'
    - 'TariffName' -> most common 'heim premium'
    - 'Ranking' -> effectively just a window function
- Geographical
    - 'City' -> Most frequent: Berlin
    - 'ZIP_code' -> extends above
- Time
    - LastUpdatedAt_UNIX -> (needs converting) just one value
- Other
    - 'ResultType' -> one value repeats "NP"

# Next Steps
- Clarifiy Data
- Enrich features (e.g. weather patterns, holidays, vacation times, geographic features, sociodemographic features...)
- Modelling
    - Meter Data: *not recommended*
    - Tarif Data: Cluster Tarifs, determine in which cluster(s) to compete
- General approach
    - Business: Clear strategy is required and be underlined by kpi's, data and technical infrastructure
    - Technical: Expand & Extend data


# Cluster Tarif Data (ex-post)

In [ ]:
# Feature Selection
def select_features(df):
    features = df[['TotalPrice_eur', 'Grundpreis_eur_per_month', 'Arbeitspreis_cent_per_kWh', 'Ranking']]
    return features

# Normalize the Features
def normalize_features(features):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    return scaled_features

# Clustering
def apply_clustering(scaled_features, n_clusters=4):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(scaled_features)
    return clusters

# Analysis
def analyze_clusters(df, clusters):
    df['Cluster'] = clusters
    cluster_analysis = df.groupby('Cluster').mean()
    return cluster_analysis

# Select relevant features
features = select_features(tarif_data)

# Normalize the features
scaled_features = normalize_features(features)

# Apply clustering
clusters = apply_clustering(scaled_features)

# Analyze clusters
cluster_analysis = analyze_clusters(tarif_data, clusters)

# Print the cluster analysis
print(cluster_analysis)

# Visualize the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=tarif_data, x='Grundpreis_eur_per_month', y='Arbeitspreis_cent_per_kWh', hue='Cluster', palette='viridis')
plt.title('Clustering of Tariff Data')
plt.xlabel('Grundpreis (EUR per month)')
plt.ylabel('Arbeitspreis (cent per kWh)')
plt.show()
